# Import

In [1]:
import os
import sys
from datetime import datetime
from datetime import timedelta as td

import pandas as pd
import plotly
import plotly.graph_objs as go
# import colorlover as cl

# import src.mongos as mg
from src.kobis import get_boxoffice

# from src.korean_holidays import hds

plotly.offline.init_notebook_mode(connected=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

# Const

In [2]:
nm_avengers1 = '어벤져스'
nm_avengers2 = '어벤져스: 에이지 오브 울트론'
nm_avengers3 = '어벤져스: 인피니티 워'
nm_avengers4 = '어벤져스: 엔드게임'
nm_myungryang = '명량'
nm_with_god = '신과함께-죄와 벌'
nm_extreme_job = '극한직업'
nm_cap_mar = '캡틴 마블'
nm_ = ''
yesterday = datetime.now() - td(days=1)
yesterday = yesterday.strftime('%-m월%-d일')
yesterday

'5월7일'

# Load

In [3]:
box = get_boxoffice()
box.head()

,audiAcc,audiChange,audiCnt,audiInten,movieCd,movieNm,openDt,rank,rankInten,rankOldAndNew,rnum,salesAcc,salesAmt,salesChange,salesInten,salesShare,scrnCnt,showCnt,targetDt,openDt_f,targetDt_f,isHoliday
0,870,100,870,870,20030191,매트릭스3 레볼루션,2003-11-05,1,0,NEW,1,7676000,7676000,100,7676000,62.6,6,16,20031111,2003-11-05,2003-11-11,False
1,177,100,177,177,20030152,위대한 유산,2003-10-24,2,0,NEW,2,1658000,1658000,100,1658000,13.5,1,3,20031111,2003-10-24,2003-11-11,False
2,116,100,116,116,20030154,황산벌,2003-10-17,3,0,NEW,3,1257000,1257000,100,1257000,10.3,1,3,20031111,2003-10-17,2003-11-11,False
3,102,100,102,102,20030247,아이덴티티,2003-10-31,4,0,NEW,4,821000,821000,100,821000,6.7,1,4,20031111,2003-10-31,2003-11-11,False
4,60,100,60,60,20030127,스캔들-조선남녀상열지사,2003-10-02,5,0,NEW,5,571000,571000,100,571000,4.7,1,3,20031111,2003-10-02,2003-11-11,False


# Functions

In [4]:
def get_movie(box, name):
    idx_movie = box['movieNm'] == name
    return box.loc[idx_movie].reset_index(drop=True)

In [5]:
def get_random_color():
    import random
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())

# Function: Graph

In [6]:
def get_hovertext(df):
    t0 = [f'{d:,} 명' for d in df['audiAcc'].astype(int).tolist()]
    t1 = [datetime.strftime(x, '%Y.%m.%d') for x in df['targetDt_f'].tolist()]
    t2 = ['휴일' if x else '평일' for x in df['isHoliday'].tolist()]
    t12 = [' '.join(x) for x in zip(t1, t2)]
    ts = [t12, t0]
    return list('<br>'.join(x) for x in zip(*ts))

In [7]:
def shorten_movie_name(movie_name):
    if movie_name == '어벤져스: 인피니티 워':
        return '어벤져스3'
    elif movie_name == '어벤져스: 에이지 오브 울트론':
        return '어벤져스2'
    elif movie_name == '어벤져스':
        return '어벤져스1'
    else:
        return movie_name

In [8]:
def get_scatter(df):
    day_passed = len(df)
#     movie_name = shorten_movie_name(df['movieNm'][0])
    movie_name = df['movieNm'][0]
    audi_acc = int(df["audiAcc"][day_passed-1])
    holiday_count = len(df.loc[(df['isHoliday'] == True)])
    return go.Scatter(
        x=df.index+1,
        y=df['audiAcc'],
        mode='lines+markers',
        name=f'{movie_name}<br>{audi_acc:,}명<br>휴일횟수: {holiday_count}일<br>',
        line=dict(
            width=2,
            color=movie_color.get(movie_name, get_random_color()),
        ),
        opacity=1 if movie_name == '어벤져스: 엔드게임' else 0.3,
        hoverinfo='text',
        hovertext=get_hovertext(df),
    )

In [9]:
def get_scatter_screen(df):
    day_passed = len(df)
#     movie_name = shorten_movie_name(df['movieNm'][0])
    movie_name = df['movieNm'][0]
    scrn_cnt = int(df["scrnCnt"][day_passed-1])
    holiday_count = len(df.loc[(df['isHoliday'] == True)])
    return go.Scatter(
        x=df.index+1,
        y=df['scrnCnt'],
        mode='lines+markers',
        name=f'{movie_name}<br>',
        line=dict(
            width=2,
            color=movie_color.get(movie_name, get_random_color()),
        ),
        opacity=1 if movie_name == '어벤져스: 엔드게임' else 0.5,
        hoverinfo='text',
        hovertext=get_hovertext(df),
    )

In [10]:
def get_layout(
   title, annotations=[], width=700, height=500,
   b=20, t=20, l=30, r=30,
   legend_ori='v', x=-0.1, y=-0.1,
   x_fixedrange=True
):
    return go.Layout(
        title=title,
        annotations=annotations,
        width=width,
        height=height,
        xaxis=dict(
            title='날짜수',
            zeroline=False,
            fixedrange=x_fixedrange,
        ),
        yaxis=dict(
            title='관객수',
            zeroline=False,
            fixedrange=True,
        ),
        margin=dict(
            b=b,
            t=t,
            l=l,
            r=r,
        ),
        legend=dict(
            orientation=legend_ori,
            font=dict(size=12),
            x=x,
            y=y,
        ),
    )

In [11]:
def get_layout_screen(
   title, annotations=[], width=700, height=500,
   b=20, t=20, l=30, r=30,
   legend_ori='v', x=-0.1, y=-0.1,
   x_fixedrange=True
):
    return go.Layout(
        title=title,
        annotations=annotations,
        width=width,
        height=height,
        xaxis=dict(
            title='날짜수',
            zeroline=False,
            fixedrange=x_fixedrange,
        ),
        yaxis=dict(
            title='스크린수',
            zeroline=False,
            fixedrange=True,
        ),
        margin=dict(
            b=b,
            t=t,
            l=l,
            r=r,
        ),
        legend=dict(
            orientation=legend_ori,
            font=dict(size=15),
            x=x,
            y=y,
        ),
    )

In [12]:
def get_textarea(text, x=-.1, y=0):
    return go.Annotation(
        text=text,
        showarrow=False,
        xref='paper',
        yref='paper',
        x=x,
        y=y,
        align='left',
        yanchor='top',
    )

# Among Avengers

## Var: Graph

In [13]:
avengers1 = get_movie(box, nm_avengers1)
avengers2 = get_movie(box, nm_avengers2)
avengers3 = get_movie(box, nm_avengers3)
avengers4 = get_movie(box, nm_avengers4)
cap_mar = get_movie(box, nm_cap_mar)

In [14]:
movie_color = {
    '어벤져스': 'green',
    '어벤져스: 에이지 오브 울트론': 'blue',
    '어벤져스: 인피니티 워': 'gold',
    '캡틴 마블': 'orange',
    '어벤져스: 엔드게임': 'red',

    '명량': 'blue',
    '신과함께-죄와 벌': 'purple',
    '극한직업': 'green',
    '국제시장': 'purple',

}

## Draw Graph

In [15]:
def draw_graph(dfs, title='', upload=False, filename='dummy'):
    data = [get_scatter(df) for df in dfs]

#     title = f"역대 어벤져스 누적관객수 비교<br>{dyp}일 차"
    layout = get_layout(
        title,
        width=1080, height=700,
        b=120, t=70, r=20, l=40,
        legend_ori='h', x=-.1, y=-.18
    )

    fig = go.Figure(data=data, layout=layout,)
    plotly.offline.iplot(fig)
    # py.iplot(fig)
    if upload:
        print('called')
        plotly.plotly.plot(fig, filename=filename, auto_open=False)

In [16]:
def draw_graph_screen(dfs, title, upload=False, filename='dummy'):
    data = [get_scatter_screen(df) for df in dfs]

#     title = f"역대 어벤져스 누적관객수 비교<br>{dyp}일 차"
    layout = get_layout_screen(
        title,
        width=1080, height=700,
        b=120, t=70, r=20, l=40,
        legend_ori='h', x=-.1, y=-.18
    )

    fig = go.Figure(data=data, layout=layout,)
    plotly.offline.iplot(fig)
    if upload:
        print('called')
        plotly.plotly.plot(fig, filename=filename, auto_open=False)

In [17]:
dfs = [avengers1, avengers2, avengers3, avengers4]  # pu,
# dyp = min(len(x) for x in dfs)
dyp = len(avengers4)
title = f'<span style="font-size:1.2em">어벤져스 시리즈<br>누적관객수<br><span style="font-size:0.8em"> ({yesterday}, {dyp}일 차)</span></span>'
# draw_graph(dfs, title,)
draw_graph(dfs, title, upload=True, filename='endgame_0')

called


# Among Top 3

In [18]:
mr = get_movie(box, nm_myungryang)
with_god = get_movie(box, nm_with_god)
extreme = get_movie(box, nm_extreme_job)

In [19]:
dfs = [mr, with_god, extreme, avengers4]
title = f'<span style="font-size:1.2em">역대 흥행 TOP3 영화<br>누적관객수<br><span style="font-size:0.8em"> ({yesterday}, {len(avengers4)}일 차)</span></span>'

# draw_graph(dfs, title,)
draw_graph(dfs, title, upload=True, filename='endgame_1')

called


# 스크린

In [20]:
dfs = [mr, extreme, avengers4, avengers3, cap_mar]
# [data.append(get_scatter(df)) for df in dfs]
# title = f"역대 흥행 TOP3 영화<br>마블 최근작 두 편<br>{len(avengers4)}일차"

title = f'<span style="font-size:1.2em">역대 흥행 TOP3 영화<br>마블 최근작 두 편 스크린 수<br><span style="font-size:0.8em">({yesterday}, {len(avengers4)}일 차)</span></span>'
# title = f'<span style="font-size:1.2em">역대 흥행 TOP3 영화<br>누적관객수<span style="font-size:0.8em"> ({len(avengers4)}일 차)</span></span>'

# draw_graph_screen(dfs, title, )
draw_graph_screen(dfs, title, upload=True, filename='endgame_2')

called
